In [16]:
!git clone https://github.com/Thomasc33/Animal-Classification.git

Cloning into 'Animal-Classification'...
remote: Enumerating objects: 5515, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 5515 (delta 3), reused 17 (delta 3), pack-reused 5498
Receiving objects: 100% (5515/5515), 656.40 MiB | 32.94 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (5409/5409), done.


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3

In [18]:
batch_size=64
data_dir = "/content/Animal-Classification/data/animals"

train_datagen = ImageDataGenerator(rescale = 1./255,
        validation_split=0.2,
        rotation_range=35,
        width_shift_range=0.25,
        preprocessing_function=tf.keras.applications.resnet.preprocess_input,
        height_shift_range=0.25,
        shear_range=0.25,
        zoom_range=0.25,
        horizontal_flip=True,
        fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)


train_generator = train_datagen.flow_from_directory(data_dir,
    target_size=(299,299),
    class_mode='categorical',
    batch_size=batch_size,
    subset = "training")
validation_generator = validation_datagen.flow_from_directory(data_dir,
    target_size=(299,299),
    class_mode='categorical',
    batch_size=batch_size,
    subset = "validation")

labels = {v: k for k, v in train_generator.class_indices.items()}

Found 4320 images belonging to 90 classes.
Found 1080 images belonging to 90 classes.


In [19]:
pre_trained_model = InceptionV3(input_shape=(299, 299, 3), include_top=True, weights='imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_output = pre_trained_model.get_layer('mixed10').output

96112376/96112376 [==============================] - 1s 0us/step


In [20]:
x = layers.Dense(1024, activation='relu')(last_output)
x = layers.Dropout(0.2)(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(90, activation='softmax')(x)
model = Model(pre_trained_model.input, x)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [22]:
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
load_model = True
if load_model:
    model = keras.models.load_model('/content/Animal-Classification/FinalModel.h5')

In [24]:
if not load_model:
    history = model.fit(train_generator,
                        validation_data=validation_generator,
                        epochs=10)

In [25]:
model.evaluate(validation_generator)

17/17 [==============================] - 239s 14s/step - loss: 0.6093 - accuracy: 0.8602


[0.6093347072601318, 0.8601852059364319]

In [26]:
#model.save('FinalModel.h5')

In [27]:
testing_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0)
testing_data = testing_datagen.flow_from_directory('/content/Animal-Classification/testing_images',
    target_size=(299,299),
    class_mode=None,
    batch_size=1,
    subset="training")

Found 1 images belonging to 1 classes.


In [28]:
np.argmax(model.predict(testing_data))

1/1 [==============================] - 2s 2s/step


10

In [29]:
labels[np.argmax(model.predict(testing_data))]

1/1 [==============================] - 0s 277ms/step


'caterpillar'

In [31]:
plt.figure(figsize=(10, 10))
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


NameError: ignored